In [ ]:
# !pip install gensim

In [9]:
!pip install Arabic-Stopwords

In [41]:
# !pip install xgboost

## Imports

In [24]:
from IPython.display import display
import pandas as pd
import arabicstopwords.arabicstopwords as stp

#vectorization
import gensim
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer

#models 
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier

In [3]:
cleaned_file_path = '../data/cleaned_dialect_dataset.xlsx'

In [4]:
pd.set_option('display.max_colwidth', None)

## Read and viewing data

In [5]:
cleaned_df = pd.read_excel(cleaned_file_path,index_col=0)

In [6]:
cleaned_df.sample(15)

,id,dialect,text,tweet_len,stem_text
285367,4.498899e+17,EG,مين اللى بيبيع الضمير ويشتري بيه الدمار حدوتة مصرية,51,مين للى يبع ضمر شتر بيه دمر حدت صرة
208129,1.136754e+18,SA,ترى جالس ابكي بدون دموع خلاص ماتحمل قلبي رهيف,67,ترى جلس ابك بدن دمع خلص ماتحمل قلب رهف
41554,1.142942e+18,LY,مايسترو بلرز مثلا مش مارة عليك,42,مايسترو لرز ثلا مش ارة علك
246095,1.158793e+18,EG,"عن الخلاف بين قادة وشباب الاخوان في تركيا حول العنف والإرهاب ما يسمونه ب ""العمل النوعي"" واللي كان ترحيل الشاب عبد الحفيظ ""أحد أعضاء تيار العنف الارهاب محمد كمال العمل النوعي"" من تركيا لمصر حلقة صارخة من حلقاته لأي شخص مكانش بايت بره مصر الشهور اللي فاتت",280,عن خلف بين قدة شبب اخو في ترك حول عنف رهب ما سمو ب `` عمل نوع '' ولل كان رحل شاب عبد حفظ `` احد عضء تير عنف رهب حمد كمل عمل نوع '' من ترك مصر حلق صرخ من حلق لأي شخص كنش بيت بره مصر شهر الل فتت
345596,7.097478e+17,KW,اول مره تختار شي صح احلام,50,اول مره خار شي صح حلم
167024,1.024334e+18,JO,الاخ الكبير مظلوم ومضطهد الأخ الكبير هو السند بعد الأب هو صمام الأمان الثاني في البيت هو اللي ممكن يتنازل عن شغلات عشان اخوانه وأخواته❤️,139,الخ كبر ظلم ضطهد لأخ كبر هو سند بعد لأب هو صمم امن ثني في بيت هو الل مكن نزل عن شغل عشن خون وأخواته❤️
48331,1.105023e+18,LY,علي العموم ابو ظبي كويسه في الصحه وممتازة للأجازات ربي يوفقهم ان شاءالله وسلم علي الشهداء اللي معاك ياصديقنا شوفلنا سيرجيو بلكي يديرلنا راي في اجازة حتي في ابو ظبي راضي بيها عادي نهارك طيب ياصديقنا,215,علي عمم ابو ظبي كوس في صحه متز اجز ربي وفق ان شاءالل سلم علي شهداء الل عاك صدق شوفل رجو بلك ديرل راي في جزة حتي في ابو ظبي رضي بيه عدي نهر طيب صدق
181155,1.098325e+18,MA,ترى مو قصدي حب حب عادي يعني متابعني اكيد ماتكرهني الا اذا عندك انفصام,80,ترى مو قصد حب حب عدي يعن تبع اكد اكر الا اذا عند فصم
359084,1.136306e+18,KW,معناته احنا صح فطرنا امس,42,عنت احن صح فطر امس
422809,1.943070e+17,AE,قــلوب الوان والـنـيه عـيـون والـبـشر مـابـيـن مـزمـار وطـبل بيـن شعـرة عـقـل وانـواع الجـنـون صـفي الـنيه يشـوفـونـه خـبـل,136,قــلوب وان ـنـ عــ ـبـشر مـابـيـن مـزمـار طـبل بيـ شعـر عـقـل ـوع جـنـ صـف ـنه يشـوفـونـه خـبـل


Apply label Encoding on our target

In [7]:
label_encoder=preprocessing.LabelEncoder ()
cleaned_df['dialect_labels'] =label_encoder.fit_transform (cleaned_df['dialect'])
cleaned_df['dialect_labels'].unique()

array([ 4,  8, 12, 11, 15, 16,  5,  9, 13, 17,  2,  3,  7,  6, 10, 14,  0,
        1])

In [11]:
cleaned_df

,id,dialect,text,tweet_len,stem_text,dialect_labels
0,1.175358e+18,IQ,لكن بالنهاية ينتفض يغير,48,لكن نهي نفض يغر,4
1,1.175416e+18,IQ,يعني هذا محسوب على البشر حيونه ووحشيه وتطلبون من الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالإرهاب,120,يعن هذا حسب على بشر حين وحش طلب من غرب حرم يؤم بدن ينع رهب,4
2,1.175450e+18,IQ,مبين من كلامه خليجي,31,بين من كلم خلج,4
3,1.175471e+18,IQ,يسلملي مرورك وروحك الحلوه,42,لمل رور ورح حله,4
4,1.175497e+18,IQ,وين هل الغيبه اخ محمد,34,وين هل غيب اخ حمد,4
...,...,...,...,...,...,...
458192,1.019485e+18,BH,مبسوطين منك اللي باسطا,37,بسط منك الل بسط,1
458193,1.021083e+18,BH,والله ماينده ابش يختي,44,ولل ايند ابش يخت,1
458194,1.017478e+18,BH,شو عملنا لك حنا تهربي مننا احنا مساكين ليش بتعملي هيك فينا,73,شو عمل لك حنا هرب ننا احن ساك ليش عمل هيك فين,1
458195,1.022430e+18,BH,الله يبارك فيها وبالعافيه,43,الل يبر فيه وبالعافيه,1


In [8]:
#one row in the x_train is null which make a problem with countvectorizer
cleaned_df.dropna(inplace=True)

### Try to remove stopwords
there is another idea to pass stopwords array to countvectorizer 

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
stopwords=set(stopwords.words("arabic"))
'لكن' in stopwords
#Contains 700 stopwords

True

In [15]:
def remove_stopwords(text):
    words=wordpunct_tokenize(text)

    filtered_sentence=[]

    for i in words:
        if i not in stopwords:
          filtered_sentence.append(i)
    return " ".join(filtered_sentence)

In [16]:
test="كتب الولد الدرس الذى كان هو ف المدرسة"
remove_stopwords(test)
#great it's works

'كتب الولد الدرس الذى المدرسة'

Apply it on cleaned text

In [17]:
cleaned_df['text_without_stopwords'] = cleaned_df.text.apply(remove_stopwords)
cleaned_df

,id,dialect,text,tweet_len,stem_text,dialect_labels,text_without_stopwords
0,1.175358e+18,IQ,لكن بالنهاية ينتفض يغير,48,لكن نهي نفض يغر,4,بالنهاية ينتفض يغير
1,1.175416e+18,IQ,يعني هذا محسوب على البشر حيونه ووحشيه وتطلبون من الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالإرهاب,120,يعن هذا حسب على بشر حين وحش طلب من غرب حرم يؤم بدن ينع رهب,4,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالإرهاب
2,1.175450e+18,IQ,مبين من كلامه خليجي,31,بين من كلم خلج,4,مبين كلامه خليجي
3,1.175471e+18,IQ,يسلملي مرورك وروحك الحلوه,42,لمل رور ورح حله,4,يسلملي مرورك وروحك الحلوه
4,1.175497e+18,IQ,وين هل الغيبه اخ محمد,34,وين هل غيب اخ حمد,4,وين الغيبه اخ محمد
...,...,...,...,...,...,...,...
458192,1.019485e+18,BH,مبسوطين منك اللي باسطا,37,بسط منك الل بسط,1,مبسوطين منك اللي باسطا
458193,1.021083e+18,BH,والله ماينده ابش يختي,44,ولل ايند ابش يخت,1,والله ماينده ابش يختي
458194,1.017478e+18,BH,شو عملنا لك حنا تهربي مننا احنا مساكين ليش بتعملي هيك فينا,73,شو عمل لك حنا هرب ننا احن ساك ليش عمل هيك فين,1,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي هيك فينا
458195,1.022430e+18,BH,الله يبارك فيها وبالعافيه,43,الل يبر فيه وبالعافيه,1,الله يبارك وبالعافيه


Apply it on stemming text 

In [26]:
cleaned_df['stem_text'] = cleaned_df.stem_text.apply(remove_stopwords)
cleaned_df

,id,dialect,text,tweet_len,stem_text,dialect_labels,text_without_stopwords
0,1.175358e+18,IQ,لكن بالنهاية ينتفض يغير,48,نهي نفض يغر,4,بالنهاية ينتفض يغير
1,1.175416e+18,IQ,يعني هذا محسوب على البشر حيونه ووحشيه وتطلبون من الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالإرهاب,120,يعن بشر وحش طلب غرب حرم يؤم بدن ينع رهب,4,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالإرهاب
2,1.175450e+18,IQ,مبين من كلامه خليجي,31,كلم خلج,4,مبين كلامه خليجي
3,1.175471e+18,IQ,يسلملي مرورك وروحك الحلوه,42,لمل رور ورح حله,4,يسلملي مرورك وروحك الحلوه
4,1.175497e+18,IQ,وين هل الغيبه اخ محمد,34,وين غيب اخ حمد,4,وين الغيبه اخ محمد
...,...,...,...,...,...,...,...
458192,1.019485e+18,BH,مبسوطين منك اللي باسطا,37,بسط منك الل بسط,1,مبسوطين منك اللي باسطا
458193,1.021083e+18,BH,والله ماينده ابش يختي,44,ولل ايند ابش يخت,1,والله ماينده ابش يختي
458194,1.017478e+18,BH,شو عملنا لك حنا تهربي مننا احنا مساكين ليش بتعملي هيك فينا,73,شو عمل حنا هرب ننا احن ساك ليش عمل هيك فين,1,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي هيك فينا
458195,1.022430e+18,BH,الله يبارك فيها وبالعافيه,43,الل يبر وبالعافيه,1,الله يبارك وبالعافيه


### Modeling
    1- working with stemming data
     
    2- working with unstemmed data (only cleaned)
     

In [9]:
cleaned_df.dialect.unique()

array(['IQ', 'LY', 'QA', 'PL', 'SY', 'TN', 'JO', 'MA', 'SA', 'YE', 'DZ',
       'EG', 'LB', 'KW', 'OM', 'SD', 'AE', 'BH'], dtype=object)

In [10]:
#a global method to prediction and print report
def print_report(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    report = metrics.classification_report(y_test, y_pred,target_names=cleaned_df.dialect.unique())
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

# Step 1 :Working with stemming data

### Split stemming data

In [29]:
X = cleaned_df['stem_text']
y = cleaned_df['dialect_labels']

# new_X = pd.Series([str (item) for item in X])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print('Number of rows of Total dataset : {}'.format(cleaned_df.shape[0]))
print('Number of rows of Train dataset : {}'.format(X_train.shape[0]))
print('Number of rows of Test dataset : {}'.format(X_test.shape[0]))

Number of rows of Total dataset : 458151
Number of rows of Train dataset : 366520
Number of rows of Test dataset : 91631


### <strong>Count Vectorizer and tfidf transformer 
- with MultinomialNB accuracy: 0.453 
- with SGDClassifier accuracy: 0.423</strong>

In [32]:
count_vec = CountVectorizer()
training_data = count_vec.fit_transform(X_train)
testing_data = count_vec.transform(X_test)

In [33]:
training_data.shape,testing_data.shape
#as seen there are 111120 words as a feature

((366520, 111120), (91631, 111120))

<strong>TfidfTransformer : using countvectorizer then tfidftransformer is equivalent to tfidfvectorizer</strong>

In [35]:
tfidf = TfidfTransformer()
training_data = tfidf.fit_transform(training_data)
testing_data = tfidf.transform(testing_data)

Naive Bayes classifier for multinomial models

In [36]:
mnb = MultinomialNB(alpha=0.08)
mnb.fit(training_data, y_train);

In [37]:
print_report(mnb, testing_data, y_test)

              precision    recall  f1-score   support

          IQ       0.40      0.27      0.32      5337
          LY       0.36      0.20      0.26      5269
          QA       0.71      0.35      0.47      3249
          PL       0.52      0.88      0.66     11595
          SY       0.81      0.35      0.49      3045
          TN       0.41      0.18      0.25      5561
          JO       0.32      0.65      0.43      8374
          MA       0.61      0.54      0.58      5562
          SA       0.55      0.59      0.57      7356
          YE       0.89      0.42      0.57      2289
          DZ       0.53      0.14      0.22      3855
          EG       0.34      0.61      0.43      8692
          LB       0.41      0.42      0.42      6204
          KW       0.40      0.31      0.35      5398
          OM       0.82      0.31      0.45      2855
          SD       0.60      0.13      0.21      3226
          AE       0.82      0.19      0.31      1754
          BH       0.63    

SGDClassifier (Linear support vector machine)

In [38]:
sgd = SGDClassifier(loss='hinge',alpha=1e-3, random_state=42)
sgd.fit(training_data, y_train);

In [39]:
print_report(sgd, testing_data, y_test)

              precision    recall  f1-score   support

          IQ       0.32      0.28      0.30      5337
          LY       0.33      0.17      0.22      5269
          QA       0.44      0.38      0.41      3249
          PL       0.49      0.87      0.63     11595
          SY       0.46      0.49      0.48      3045
          TN       0.31      0.18      0.22      5561
          JO       0.42      0.46      0.44      8374
          MA       0.47      0.62      0.54      5562
          SA       0.48      0.56      0.52      7356
          YE       0.54      0.51      0.52      2289
          DZ       0.29      0.19      0.23      3855
          EG       0.41      0.37      0.39      8692
          LB       0.38      0.41      0.40      6204
          KW       0.36      0.24      0.29      5398
          OM       0.41      0.28      0.33      2855
          SD       0.33      0.15      0.21      3226
          AE       0.43      0.31      0.36      1754
          BH       0.18    

# Step2 : working with cleaned data only

### Split unstem data

In [18]:
X = cleaned_df['text']
y = cleaned_df['dialect_labels']

# new_X = pd.Series([str (item) for item in X])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print('Number of rows of Total dataset : {}'.format(cleaned_df.shape[0]))
print('Number of rows of Train dataset : {}'.format(X_train.shape[0]))
print('Number of rows of Test dataset : {}'.format(X_test.shape[0]))

Number of rows of Total dataset : 458151
Number of rows of Train dataset : 366520
Number of rows of Test dataset : 91631


### <strong>Count vectorizer
- with MultinomialNB, accuracy:  0.554 <strong>Is the best result I got</strong> 
- with SGD, accuracy: 0.490 </strong> 

In [19]:
count_vec = CountVectorizer()
training_data = count_vec.fit_transform(X_train)
testing_data = count_vec.transform(X_test)

In [20]:
training_data.shape,testing_data.shape
#as seen there are 436338 words as a feature

((366520, 436338), (91631, 436338))

In [21]:
mnb = MultinomialNB(alpha=0.08)
mnb.fit(training_data, y_train);

In [22]:
print_report(mnb, testing_data, y_test)

              precision    recall  f1-score   support

          IQ       0.42      0.44      0.43      5337
          LY       0.35      0.35      0.35      5269
          QA       0.65      0.57      0.61      3249
          PL       0.73      0.88      0.80     11595
          SY       0.70      0.55      0.61      3045
          TN       0.44      0.35      0.39      5561
          JO       0.48      0.57      0.52      8374
          MA       0.67      0.68      0.67      5562
          SA       0.73      0.70      0.71      7356
          YE       0.80      0.62      0.69      2289
          DZ       0.45      0.33      0.38      3855
          EG       0.47      0.59      0.53      8692
          LB       0.45      0.51      0.48      6204
          KW       0.42      0.45      0.43      5398
          OM       0.77      0.57      0.66      2855
          SD       0.49      0.31      0.38      3226
          AE       0.70      0.45      0.54      1754
          BH       0.44    

In [25]:
sgd = SGDClassifier(loss='hinge',alpha=1e-3, random_state=42) #loss=hinge which gives a linear svm
sgd.fit(training_data, y_train);

In [26]:
print_report(sgd, testing_data, y_test)

              precision    recall  f1-score   support

          IQ       0.42      0.36      0.39      5337
          LY       0.33      0.24      0.28      5269
          QA       0.53      0.49      0.51      3249
          PL       0.57      0.88      0.69     11595
          SY       0.51      0.56      0.53      3045
          TN       0.42      0.23      0.29      5561
          JO       0.47      0.56      0.51      8374
          MA       0.51      0.68      0.58      5562
          SA       0.58      0.64      0.61      7356
          YE       0.57      0.55      0.56      2289
          DZ       0.34      0.29      0.31      3855
          EG       0.49      0.43      0.46      8692
          LB       0.46      0.45      0.45      6204
          KW       0.42      0.38      0.40      5398
          OM       0.58      0.41      0.48      2855
          SD       0.38      0.21      0.27      3226
          AE       0.63      0.39      0.48      1754
          BH       0.23    

### <strong> Tfidf vecotrizer 
- with MultinomialNB accuracy: 0.524
- with SGD, accuracy: 0.504
</strong>

In [27]:
vectorizer = TfidfVectorizer(sublinear_tf=True)
training_data = vectorizer.fit_transform(X_train)
testing_data = vectorizer.transform(X_test)

In [28]:
training_data.shape,testing_data.shape
#as seen there are 436338 words as a feature

((366520, 436338), (91631, 436338))

In [29]:
mnb = MultinomialNB(alpha=0.08)
mnb.fit(training_data, y_train)

MultinomialNB(alpha=0.08)

In [30]:
print_report(mnb, testing_data, y_test)

              precision    recall  f1-score   support

          IQ       0.46      0.38      0.41      5337
          LY       0.39      0.27      0.32      5269
          QA       0.79      0.46      0.58      3249
          PL       0.61      0.92      0.73     11595
          SY       0.85      0.40      0.55      3045
          TN       0.46      0.27      0.34      5561
          JO       0.38      0.68      0.49      8374
          MA       0.69      0.64      0.66      5562
          SA       0.66      0.70      0.68      7356
          YE       0.94      0.52      0.67      2289
          DZ       0.61      0.20      0.30      3855
          EG       0.39      0.66      0.49      8692
          LB       0.44      0.51      0.47      6204
          KW       0.46      0.39      0.42      5398
          OM       0.93      0.39      0.55      2855
          SD       0.69      0.16      0.25      3226
          AE       0.95      0.25      0.40      1754
          BH       0.77    

In [31]:
sgd = SGDClassifier(loss='hinge',alpha=1e-3, random_state=42)
sgd.fit(training_data, y_train);

In [32]:
print_report(sgd, testing_data, y_test)

              precision    recall  f1-score   support

          IQ       0.45      0.38      0.41      5337
          LY       0.41      0.25      0.31      5269
          QA       0.56      0.49      0.52      3249
          PL       0.52      0.91      0.66     11595
          SY       0.56      0.56      0.56      3045
          TN       0.46      0.24      0.32      5561
          JO       0.48      0.57      0.53      8374
          MA       0.53      0.71      0.60      5562
          SA       0.57      0.66      0.61      7356
          YE       0.62      0.57      0.59      2289
          DZ       0.42      0.29      0.34      3855
          EG       0.50      0.46      0.48      8692
          LB       0.45      0.47      0.46      6204
          KW       0.46      0.36      0.41      5398
          OM       0.57      0.38      0.45      2855
          SD       0.45      0.20      0.27      3226
          AE       0.58      0.40      0.47      1754
          BH       0.37    

# Neural Network 
- incompleted (since I have issue with my laptop resource, I need to make hyperparamter tuning to get better) accuracy :0.3750477433204651

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
from tensorflow.keras.utils import to_categorical

In [8]:
X = cleaned_df['text']
y = cleaned_df['dialect_labels']

# new_X = pd.Series([str (item) for item in X])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print('Number of rows of Total dataset : {}'.format(cleaned_df.shape[0]))
print('Number of rows of Train dataset : {}'.format(X_train.shape[0]))
print('Number of rows of Test dataset : {}'.format(X_test.shape[0]))

Number of rows of Total dataset : 458151
Number of rows of Train dataset : 366520
Number of rows of Test dataset : 91631


In [24]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [25]:
tokenize.fit_on_texts(X_train) # only fit on train
x_train = tokenize.texts_to_matrix(X_train)
x_test = tokenize.texts_to_matrix(X_test)

Note : i have some problems with my laptop - MemoryError: Unable to allocate 13.7 GiB for an array with shape (366520, 5000) and data type float64

In [26]:
input_shape = x_train.shape[1]

In [27]:
model = keras.Sequential()
model.add(keras.layers.Dense(16,input_shape=(input_shape,), activation='relu'))
model.add(Dropout(0.5))
model.add(keras.layers.Dense(18,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                16016     
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 18)                306       
                                                                 
Total params: 16,322
Trainable params: 16,322
Non-trainable params: 0
_________________________________________________________________


In [28]:
y_train = to_categorical(y_train, 18)
y_test = to_categorical(y_test, 18)

In [29]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (366520, 1000)
x_test shape: (91631, 1000)
y_train shape: (366520, 18, 18)
y_test shape: (91631, 18, 18)


In [30]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=2,
                    verbose=1,
                    validation_split=0.2)

Epoch 1/2


ValueError: in user code:

    File "C:\Users\Omar Wael\Anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Omar Wael\Anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Omar Wael\Anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Omar Wael\Anaconda3\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\Omar Wael\Anaconda3\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\Omar Wael\Anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample

In [ ]:
score = model.evaluate(x_test, y_test,
                       batch_size=32, verbose=1)
print('Test accuracy:', score[1])

2864/2864 [==============================] - 23s 8ms/step - loss: 1.9636 - accuracy: 0.3750
Test accuracy: 0.3750477433204651


# Prepare best result for deployment
- Count vectorizer with MutlinomialNB

In [33]:
X = cleaned_df['text']
y = cleaned_df['dialect_labels']

# new_X = pd.Series([str (item) for item in X])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print('Number of rows of Total dataset : {}'.format(cleaned_df.shape[0]))
print('Number of rows of Train dataset : {}'.format(X_train.shape[0]))
print('Number of rows of Test dataset : {}'.format(X_test.shape[0]))

Number of rows of Total dataset : 458151
Number of rows of Train dataset : 366520
Number of rows of Test dataset : 91631


In [34]:
pipeline = Pipeline(steps= [('vec', CountVectorizer()),
                            ('model', MultinomialNB(alpha=0.08))])
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vec', CountVectorizer()),
                ('model', MultinomialNB(alpha=0.08))])

In [35]:
print_report(pipeline,X_test,y_test)

              precision    recall  f1-score   support

          IQ       0.42      0.44      0.43      5337
          LY       0.35      0.35      0.35      5269
          QA       0.65      0.57      0.61      3249
          PL       0.73      0.88      0.80     11595
          SY       0.70      0.55      0.61      3045
          TN       0.44      0.35      0.39      5561
          JO       0.48      0.57      0.52      8374
          MA       0.67      0.68      0.67      5562
          SA       0.73      0.70      0.71      7356
          YE       0.80      0.62      0.69      2289
          DZ       0.45      0.33      0.38      3855
          EG       0.47      0.59      0.53      8692
          LB       0.45      0.51      0.48      6204
          KW       0.42      0.45      0.43      5398
          OM       0.77      0.57      0.66      2855
          SD       0.49      0.31      0.38      3226
          AE       0.70      0.45      0.54      1754
          BH       0.44    

In [36]:
from joblib import dump

dump(pipeline, filename="best_model.joblib")

['best_model.joblib']